In [1]:
%%html
<style>
    .dothis{
    font-weight: bold;
    color: #ff7f0e;
    font-size:large
    }
</style>

# Glacier mapping by means of band math

*****

This notebook demonstrates how we can use Python to do the same glacier mapping that we earlier carried out in QGIS.

There are several points at which you will need to modify a line of code. These are identified by <span class='dothis'>bold orange text</span>.


### Reading the data

We start with a few preparation such as reading all the packages we use in the processing.

In [2]:
%load_ext autoreload
%autoreload 2


import time
import psutil
import dask.distributed
import rioxarray
import numpy as np
import xarray as xr
from pystac_client import Client
import matplotlib.pyplot as plt
import pandas as pd 
from sdc_utilities import get_alias_band, load_product_ts

# silence warning (not recommended during development)
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# Change the size of the figures displayed in the notebooks: units are inches, order is (width, height).
plt.rcParams['figure.figsize'] = (16,8)  

# Initiate Dask Env
client = dask.distributed.Client()
# display(client)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'pystac_client'

**The next cell contains the dataset configuration information, which we have already made for you using the [config_tool-SwissDC](config_tool-SwissDC.ipynb) notebook. This ensures that you will have data well suited to map glacier extent.**

**To make this notebook run you will need to use anyone of Landsat or Sentinel 2 products with the red green and blue bands, as well as at least one swir band.**

In [ ]:

product = 'landsat_ot_c2_l2'
measurements = ['QA_PIXEL', 'blue', 'green', 'red', 'nir', 'swir_1', 'swir_2']
get_alias_band(product, measurements, measurements_file='measurements_SwissDC.csv')

In [ ]:
# Configuration
catalog = Client.open("https://explorer.swissdatacube.org/stac")

product = 'landsat_ot_c2_l2'

# you can use either specify the real measurement names or the aliases (https://explorer.swissdatacube.org/products/landsat_ot_c2_l2)
aliases = ['QA_PIXEL', 'blue', 'green', 'red', 'nir', 'swir_1', 'swir_2']
measurements, aliases = get_alias_band(product,measurements, measurements_file="measurements_SwissDC.csv")

longitude = (7.73228, 7.957461)
latitude = (45.877007, 46.022142)
crs = 'epsg:4326'

time = ('2016-08-01', '2016-09-30')
# the following date formats are also valid:
# time = ('2000-01-01', '2001-12-31')
# time=('2000-01', '2001-12')
# time=('2000', '2001')

output_crs = 'epsg:2056'
resolution = -30.0, 30.0


****

__In the following we are using `load_product_ts()` to load the data. There is also a function to mask out cloud obscured values using the pixel quality bands (`QA_PIXEL` in Landsat and `SLC` in Sentinel). We see later more on this.__

Here we are using `load_product_ts` because the cloud masking does not work in the mountains where snow and ice get confused with clouds. 

**However, this means we now have to look at the scenes and manually decide which ones are cloud free.**

****

In [ ]:
dataset_in = load_product_ts(catalog = catalog,
                          product = product,
                          time = time,
                          longitude = longitude,
                          latitude = latitude,
                          measurements = measurements,
                          output_crs = output_crs,
                          resolution = resolution,
                          rename = True,               # This will overwrite the band names with the aliases names ...
                          alias_names = aliases,        # ... that need to be specified here in that case.
                          )

In [ ]:
dataset_in

In [ ]:
# plot the xarray.Dataset to get an overview of the different scenes along the time axis
dataset_in.blue.plot(col='time', col_wrap=5, cmap='Greys', vmin=0)

### Selecting a cloud free scene

As mentioned before, we have used the `dc.load()` function which does not mask cloud covered areas. 

<span class='dothis'>We now have to plot all time steps again and then visually define which of the scenes is cloud free.</span>

This is best done by visualizing composites, either true color or false color. In the following we first visualize the data in true color, followed by two examples of popular false color visualization for Landsat data.

In [ ]:
# Let's plot composites in True color (red, green, blue)
# robust=True guesses the minimum and maximum values for each image.
dataset_in[['red','green','blue']].to_array().plot.imshow(col='time',col_wrap=5, robust=True)

In [ ]:
# Let's plot composites in False color (nir, red, green)
dataset_in[['nir','red', 'green']].to_array().plot.imshow(col='time',col_wrap=5, robust=True)

In [ ]:
# Let's plot composites in another false color combination, this time (swir1, nir, red) which is well suited to see ice and snow
dataset_in[['swir_1','nir', 'red']].to_array().plot.imshow(col='time',col_wrap=5, robust=True)

Regardless of which visualization we work with, we see that from the seven scenes, only one appears cloud free: **2016-08-25T10:10:52**. We continue working only with that scene.

In [ ]:
# Let's select the image we are interested in and save it to a new variable.
# To do this we provide the date of the image we selected above 
# The squeeze() command removes the time dimension from the DataArray, so you are left with two dimensions: latitude and longitude.
mosaic = dataset_in.sel(time='2016-08-25').squeeze()
mosaic

<a id='mosaic_plot'> </a>
Plot mosaic the default way.

In [ ]:

# To do this we need to provide the list of bands we are interested in.
mosaic[['red','green','blue']].to_array().plot.imshow(robust=True)
plt.gca().set_aspect('equal')

In [ ]:
mosaic

In [ ]:
# Now export to a GeoTIFF file.
mosaic[['red','green','blue']].astype('int32').rio.to_raster("glaciers.tif", driver="COG")



### Band math for glacier detection

*****

We now detect glacier surfaces using the band math: *G = Red / Swir*

Thereby a threshold needs to be defined to distinguish glaciers from non-glacierized terrain.

In [ ]:
G = mosaic.red / mosaic.swir_1

In [ ]:
G.plot()

We can (or can not) recognize the result. Maybe we can improve the extent of the automatically generated scale. 

It seems that inside the dataset there might be one or a few pixels with extreme values (either artifacts or values indicating invalid pixels). These outliers result in the automated choice of the data range. 

The easy way to visualize the data while ignoring the outliers is to pass the parameter robust=True. This will use the 2nd and 98th percentiles of the data to compute the color limits.

In [ ]:
G.plot(robust=True)

**Note how the color palette has changed.** This is simply because the `DataArray.plot()` function we are using tries to automatically define the optimal colormap. 

We can suppress this behaviour by specifying a colormap from https://matplotlib.org/stable/gallery/color/colormap_reference.html

In [ ]:
G.plot(robust=True, cmap='bwr')

Now let's try to plot the data as a histogram

In [ ]:
G.plot.hist()

If there would be an outlier, this would either throw an error message, telling us that the data range reaches infinity. Other problems could be (if you load the dataset with the argument `scale_offset` that some division causes artifacts with e.g. negative values. In the text output (directly below the executed cell), you can see the bin breaks. If you have negative values they will show up there.

In case you will encounter the infinity error or strange automatic bin breaks, there is a solution below. 

- First, remove all values that have the value "infinte" and then try to plot the histogram again.
- Keep only values bigger `x_low` (`x_low` would be your lower threshold value)
- Keep only values small `x_hig` (`x_hig` would be your upper threshold value)

In [ ]:
G = G.where(np.isfinite(G))  # Set all values that are not finite to NaN (Not a Number)
G = G.where((G>=0) & (G<=100),np.nan)  # Set all values that are not finite to NaN (Not a Number)

In [ ]:
G.plot.hist()

This works but there are situation where we might want to have more control on how exactly a histogram is plotted. For example: 

* Let's change the range of values shown in the histogram of G
* Let's increase the number of bins in the histogram

In [ ]:
G.plot.hist(bins=40, range=(-50, 50))  # We added the "bin" keyword that allows specifying the number of bins to be used

This is not perfect yet, e.g. there is no need to visualize all the absence of data on the left-hand side. <span class='dothis'>Adjust the parameters in the plotting command above so you achieve an optimal visualization.</span>

### Selecting the threshold value for glacier mapping

Now to the actual glacier mapping. <span class='dothis'>As we did in QGIS, we will define a threshold to distinguish between glaciers (where <i>G > threshold</i>) and non-glacierized terrain (<i>G < threshold</i>).</span>

We can directly test whether the threshold is appropriate by plotting the scene, but this time telling the program to use discrete colors for all values that are within intervals that we define with the `levels` keyword. Note that we need to define at least two values for `levels`. We simply chose the first one very low, outside the range of data values.



In [ ]:
# here we set the threshold - CHANGE THIS TO VARY THE THRESHOLD!
threshold = 1

In [ ]:
# Using the "levels" keyword which allows specifying the boundaries between discrete classes for plotting. 
G.plot(levels=[0, threshold])  # Here the resulting classes are (1) smaller -100, (2) between -100 and "threshold", (3) larger than "Trheshold"

This plot now shows all grid cell with values between -100 and "threshold" in one colour, all grid cells with values > threshold in another colour.

This is excellent, **but has the disadvantage that we have no comparison with the original scene, to be able to assess how accurately our chosen threshold distinguishes glaciers from non-glacierized terrain.**

We thus go more fancy and **plot the above threshold map *and* a false color composite of the original scene on top of each other.** We do this by simply calling the `plot()` function twice, the two graphics will then be plotted into the same figure. Furthermore, we use the `alpha` parameter to define how transparent the threshold map should be (`alpha = 0` fully transparent, `alpha = 1` fully opaque).

In [ ]:
G.plot(levels=[0, threshold], alpha=0.3)
mosaic[['red','green','blue']].to_array().plot.imshow(robust=True, alpha=0.3)

With these tools at hand, <span class='dothis'>vary the threshold to find an optimal treshold value.</span>

### Basic statistics

**Now to some very basic statistics: How many percent of the chosen satellite image are glacier covered?**

We calculate this by taking the number of glacierized grid cells `(G.where(G > threshold)).count()` and divide them by the total number of grid cells `G.count()`. The result is again a xr.DataArray which contains just the value.

In [ ]:
100 * (G.where(G > threshold)).count() / G.count()

In [ ]:
(100 * (G.where(G > threshold)).count() / G.count()).values     # adding .values to an xarray returns just the values of the array inside it --> 41.9 %

### Export the results

After we found an optimal threshold value, let's now create an array where all glacier grid cells have the value 1, and all non-glacierized cells have the value 0.

We do this by using the `xr.where()` function. Its syntax is `xr.where(condition, value to use where condition is True, value to use where condition is False)`.

In [ ]:
G

In [ ]:
glacier = xr.where(G > threshold, 1, 0)
glacier

In [ ]:
glacier.plot()  # plt it once again to see whether the result is what we want

<span class='dothis'>If everything looks fine, then it is now time to export our glacier map. We do this by using the function `rio.to_raster()`.</span>

Note that our xr.DataArray named *glacier* is of the data type `int32` (= 32 bit integer). The export function cannot handle this data type. Thus we convert *glacier* to data type `int16` (16 bit integer) during export by using `glacier.astype('int16')`.

This makes a 'GeoTIFF' file which can be loaded into software like QGIS or ArcGIS. Note that GeoTIFFs are not properly understood by programs like Photos or Photoshop, so they will look quite strange if they open at all!

In [ ]:
glacier.astype('int16').rio.to_raster("glacier_map.tif")

In [ ]:
client.close()